In [17]:
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numba as nb
from copy import copy

### Age profile of income 

In [18]:
## some life cycle paras 
T = 40
L = 60

T_q = T*4
L_q = L*4

In [19]:
def y2q_interpolate(xs_y):
    
    """
    this function turns an array of yealry rates into quarterly rates via linear interpolation 
    """

    n_y = len(xs_y)
    y_id = np.arange(n_y)
    q_id = y_id*4

    n_q = 4*n_y
    xs_q = np.empty(n_q)
    
    for i_y in range(n_y):
        xs_q[4*i_y] = xs_y[i_y]
        for i_q in np.arange(1,4):
            q_id_this = 4*i_y+i_q
            xs_q[q_id_this] = np.interp(q_id_this,q_id,xs_y)
            
    return xs_q

In [20]:
age_profile_data ='SCF'

if age_profile_data=='SIPP':
    ## import age income profile 

    age_profile = pd.read_stata('../OtherData/age_profile.dta')   

    ## select age range for the model and turn it into an array 
    lc_wages = np.array(age_profile[(age_profile['age']>=24) &(age_profile['age']<=64)]['wage_age'])
    #print(str(len(lc_wages)),'years since age 25')

    ## growth rates since initial age over life cicle before retirement
    lc_G = lc_wages[1:]/lc_wages[:-1]

    T = 40
    L = 60
    ## growth rates after retirement

    lc_G_rt = np.ones(L-T)
    lc_G_rt[0] = 0.6


    lc_G_full = np.concatenate([lc_G,lc_G_rt])
    #lc_G_full = np.ones_like(lc_G_full)

    
    ## turn yearly number to quarterly number with interpolation 

    ## get the quarterly income profile over life cycle before retirement 
    lc_G_q = y2q_interpolate(lc_G)

    lc_G_q_rt = y2q_interpolate(lc_G_rt)

    lc_G_q_full = np.concatenate([lc_G_q,lc_G_q_rt])
    
    
elif age_profile_data=='SCF':
    ## import age income profile 
    SCF_profile = pd.read_pickle('data/SCF_age_profile.pkl')
    SCF_profile = SCF_profile[(SCF_profile.index>=24) & (SCF_profile.index<=85)]
    lc_p_incom = np.exp(np.array(SCF_profile['av_lnorminc']))
    lc_G_full = lc_p_incom[1:]/lc_p_incom[:-1] 
    assert len(lc_G_full) == L,'length of G needs to be equal to L'
    lc_G_q_full = y2q_interpolate(lc_G_full)


### Income risk estimates 

In [21]:
risks_est = pd.read_stata('../OtherData/sipp/sipp_history_vol_decomposed.dta')
## risks of permanent and transitory component 

σ_ψ_q_sipp = np.sqrt(risks_est['permanent']**2*3)
σ_θ_q_sipp = np.sqrt(risks_est['permanent']**2/3)

## p/t ratio 
kappas_sipp  = risks_est['permanent']/risks_est['transitory']
kappa_sipp = np.median(kappas_sipp.dropna())

In [22]:
## risks of permanent and transitory component 

σ_ψ_q_sipp = np.sqrt(risks_est['permanent']**2*3)
σ_θ_q_sipp = np.sqrt(risks_est['permanent']**2/3)

## p/t ratio 
kappas_sipp  = risks_est['permanent']/risks_est['transitory']
kappa_sipp = np.median(kappas_sipp.dropna())

### Initial conditions

In [23]:
from statsmodels.stats.weightstats import DescrStatsW

## SCF data 
SCF2016 = pd.read_stata('rscfp2016.dta')
SCF2016 = SCF2016[SCF2016['age']==25]
SCF2016 = SCF2016.drop_duplicates(subset=['yy1'])
SCF2016 = SCF2016[['norminc','networth','wgt']]
SCF2016 = SCF2016[SCF2016['norminc']>0]

## permanent income at age 25 
pinc_SCF = SCF2016[['norminc','wgt']]
lpinc_SCF_pos, lpinc_SCF_wgt = np.log(pinc_SCF['norminc']),np.array(pinc_SCF['wgt'])

## wealth 
b_SCF = SCF2016[['networth','wgt']]
lb_SCF_pos, lb_SCF_wgt = b_SCF['networth'],np.array(b_SCF['wgt'])

## wealth to permanent income
SCF2016['networth2pinc'] = SCF2016['networth']/SCF2016['norminc']
b_SCF, b_wgt = SCF2016['networth2pinc'], SCF2016['wgt']

## compute data moments 
σ_ψ_init_SCF = DescrStatsW(lpinc_SCF_pos, weights=lpinc_SCF_wgt, ddof=1).std
σ_b_init_SCF = DescrStatsW(lb_SCF_pos, weights=lb_SCF_wgt, ddof=1).std
b_SCF = DescrStatsW(b_SCF,weights=b_wgt, ddof=1).mean ## annual 
b_q_SCF = b_SCF*4  ## quarterly  

### subjective profile estiamtion 

In [24]:
## import subjective profile estimation results 
SCE_est_q = pd.read_pickle('data/subjective_profile_est_q.pkl')
SCE_est_y = pd.read_pickle('data/subjective_profile_est_y.pkl')
SCE_est_q = SCE_est_q['baseline']
SCE_est_y = SCE_est_y['baseline']

In [25]:
## create a dictionary of parameters 
life_cycle_paras_q = {'ρ': 2.0, 
                    'β': 0.98**(1/4), 
                    'P': np.array([[0.18, 0.82],
                                   [0.04, 0.96]]), 
                    'z_val': np.array([0., 1.]), 
                    'σ_ψ': np.sqrt(0.15**2*4/11), 
                    'σ_θ': np.sqrt(0.1**2*4), 
                    'U': 0.0, 
                    'LivPrb': 1.0-0.00625, 
                    'R': 1.01**(1/4), 
                    'W': 1.0, 
                    'T': T_q, 
                    'L': L_q, 
                    'G':lc_G_q_full, 
                    'unemp_insurance': 0.15, 
                    'pension': 1.0, 
                    'σ_ψ_init': σ_ψ_init_SCF, 
                    'init_b': b_q_SCF, 
                    'λ': 0.0, 
                    'λ_SS': 0.0, 
                    'transfer': 0.0, 
                    'bequest_ratio': 0.0,
                    'κ':kappa_sipp,
                    
                    ## subjective profile
                    'q':SCE_est_q.loc['$q$'],
                    'p':SCE_est_q.loc['$p$'],
                    'σ_ψ_2mkv':np.array([SCE_est_q.loc['$\tilde\sigma^l_\psi$'],
                                       SCE_est_q.loc['$\tilde\sigma^h_\psi$']]),
                    'σ_θ_2mkv':np.array([SCE_est_q.loc['$\tilde\sigma^l_\theta$'],
                                       SCE_est_q.loc['$\tilde\sigma^h_\theta$']]),
                    'mho_2mkv':np.array([SCE_est_q.loc['$\tilde \mho^l$'],
                                         SCE_est_q.loc['$\tilde \mho^h$']]),
                    'E_2mkv':np.array([SCE_est_q.loc['$\tilde E^l$'],
                                      SCE_est_q.loc['$\tilde E^h$']])
                }

In [26]:
life_cycle_paras_q

{'ρ': 2.0,
 'β': 0.9949620563926881,
 'P': array([[0.18, 0.82],
        [0.04, 0.96]]),
 'z_val': array([0., 1.]),
 'σ_ψ': 0.09045340337332908,
 'σ_θ': 0.2,
 'U': 0.0,
 'LivPrb': 0.99375,
 'R': 1.0024906793143211,
 'W': 1.0,
 'T': 160,
 'L': 240,
 'G': array([1.10455031, 1.14177438, 1.17899846, 1.21622253, 1.2534466 ,
        1.2391538 , 1.22486101, 1.21056821, 1.19627542, 1.11529279,
        1.03431016, 0.95332754, 0.87234491, 0.94050292, 1.00866093,
        1.07681894, 1.14497695, 1.09103184, 1.03708673, 0.98314161,
        0.9291965 , 0.96467321, 1.00014991, 1.03562661, 1.07110332,
        1.05767311, 1.0442429 , 1.0308127 , 1.01738249, 1.00710306,
        0.99682364, 0.98654421, 0.97626478, 1.01450601, 1.05274724,
        1.09098846, 1.12922969, 1.09677595, 1.06432222, 1.03186848,
        0.99941475, 0.99293505, 0.98645536, 0.97997567, 0.97349597,
        1.0024493 , 1.03140263, 1.06035597, 1.0893093 , 1.06286696,
        1.03642461, 1.00998227, 0.98353993, 1.02488656, 1.06623319,


In [27]:
## create a dictionary of parameters 
life_cycle_paras_y = {'ρ': 2.0, 
                    'β': 0.98, 
                    'P': np.array([[0.18, 0.82],
                                   [0.04, 0.96]]), 
                    'z_val': np.array([0., 1.]), 
                    'σ_ψ': np.sqrt(0.15**2), 
                    'σ_θ': np.sqrt(0.1**2), 
                    'U': 0.0, 
                    'LivPrb': 1.0-0.00625, 
                    'R': 1.01, 
                    'W': 1.0, 
                    'T': T, 
                    'L': L, 
                    'G':lc_G_full, 
                    'unemp_insurance': 0.15, 
                    'pension': 1.0, 
                    'σ_ψ_init': σ_ψ_init_SCF, 
                    'init_b': b_SCF, 
                    'λ': 0.0, 
                    'λ_SS': 0.0, 
                    'transfer': 0.0, 
                    'bequest_ratio': 0.0,
                    'κ':kappa_sipp,
                    
                    ## subjective profile
                    'q':SCE_est_y.loc['$q$'],
                    'p':SCE_est_y.loc['$p$'],
                    'σ_ψ_2mkv':np.array([SCE_est_y.loc['$\tilde\sigma^l_\psi$'],
                                       SCE_est_y.loc['$\tilde\sigma^h_\psi$']]),
                    'σ_θ_2mkv':np.array([SCE_est_y.loc['$\tilde\sigma^l_\theta$'],
                                       SCE_est_y.loc['$\tilde\sigma^h_\theta$']]),
                    'mho_2mkv':np.array([SCE_est_y.loc['$\tilde \mho^l$'],
                                         SCE_est_y.loc['$\tilde \mho^h$']]),
                    'E_2mkv':np.array([SCE_est_y.loc['$\tilde E^l$'],
                                      SCE_est_y.loc['$\tilde E^h$']])
                }

In [28]:
life_cycle_paras_y

{'ρ': 2.0,
 'β': 0.98,
 'P': array([[0.18, 0.82],
        [0.04, 0.96]]),
 'z_val': array([0., 1.]),
 'σ_ψ': 0.15,
 'σ_θ': 0.1,
 'U': 0.0,
 'LivPrb': 0.99375,
 'R': 1.01,
 'W': 1.0,
 'T': 40,
 'L': 60,
 'G': array([1.10455031, 1.2534466 , 1.19627542, 0.87234491, 1.14497695,
        0.9291965 , 1.07110332, 1.01738249, 0.97626478, 1.12922969,
        0.99941475, 0.97349597, 1.0893093 , 0.98353993, 1.14892645,
        0.97906461, 0.91025961, 1.04760429, 1.17548467, 0.85318063,
        1.17776868, 0.90491453, 1.09848218, 0.97082377, 0.87554249,
        1.02371263, 0.91572759, 1.11870388, 0.90662006, 0.89395724,
        1.01534996, 1.22118921, 0.75860141, 1.42282798, 0.82526106,
        1.19399751, 0.82243583, 0.98492788, 0.95760787, 0.81536932,
        1.31880938, 0.78579426, 0.97751472, 1.4357771 , 0.71825979,
        0.94115225, 1.18988227, 0.92526562, 0.91878387, 1.00682059,
        1.01459173, 0.87216358, 1.08948598, 1.03773095, 1.32181813,
        0.7292744 , 1.08935167, 0.75156075, 1

In [29]:
#with open("parameters.txt", "wb") as fp:
#    pickle.dump(life_cycle_paras, fp)

### Export the parameters into a table used in the draft

In [84]:
life_cycle_paras_y_copy = copy(life_cycle_paras_y)

In [85]:
del life_cycle_paras_y_copy['G']  
del life_cycle_paras_y_copy['σ_ψ_2mkv']  
del life_cycle_paras_y_copy['σ_θ_2mkv']  
del life_cycle_paras_y_copy['mho_2mkv']  
del life_cycle_paras_y_copy['E_2mkv']  
del life_cycle_paras_y_copy['P']  
del life_cycle_paras_y_copy['z_val']  
del life_cycle_paras_y_copy['U']  
del life_cycle_paras_y_copy['κ']  
del life_cycle_paras_y_copy['q']  
del life_cycle_paras_y_copy['p']  


## rewrite some parameters' names

life_cycle_paras_y_copy['U2U'] = life_cycle_paras_y['P'][0,0]
life_cycle_paras_y_copy['E2E'] = life_cycle_paras_y['P'][1,1]

## rename some 
life_cycle_paras_y_copy['1-D'] =  life_cycle_paras_y_copy.pop('LivPrb')
life_cycle_paras_y_copy['μ'] =  life_cycle_paras_y_copy.pop('unemp_insurance')


## rounding 
life_cycle_paras_y_copy['1-D'] = round(life_cycle_paras_y_copy['1-D'],3)
life_cycle_paras_y_copy['σ_ψ_init'] = round(life_cycle_paras_y_copy['σ_ψ_init'],3)
life_cycle_paras_y_copy['init_b'] = round(life_cycle_paras_y_copy['init_b'],3)

In [77]:
## turn scalor to list 

#for key in life_cycle_paras_y_copy.keys():
#    life_cycle_paras_y_copy[key] = [life_cycle_paras_y_copy[key]]

In [86]:
life_cycle_paras_y_copy

{'ρ': 2.0,
 'β': 0.98,
 'σ_ψ': 0.15,
 'σ_θ': 0.1,
 'R': 1.01,
 'W': 1.0,
 'T': 40,
 'L': 60,
 'pension': 1.0,
 'σ_ψ_init': 0.629,
 'init_b': 0.554,
 'λ': 0.0,
 'λ_SS': 0.0,
 'transfer': 0.0,
 'bequest_ratio': 0.0,
 'U2U': 0.18,
 'E2E': 0.96,
 '1-D': 0.994,
 'μ': 0.15}

In [94]:
# making blocks 

blocknames =['risk',
            'initial condition',
            'life-cycle',
            'preference',
            'policy',
            'production']

prefernece = ['ρ','β']
lifecycle = ['T','L','1-D']
risk  = ['σ_ψ','σ_θ','U2U','E2E']
initial = ['σ_ψ_init','init_b','bequest_ratio']
policy = ['μ','λ','λ_SS','transfer']

block_all= [risk,
            initial,
            lifecycle,
            prefernece,
            policy]

## create multiple layer dictionary 

life_cycle_paras_y_copy_ml = {}

for i,block in enumerate(block_all):
    life_cycle_paras_y_copy_ml[blocknames[i]] =  {k: v for k, v 
                                               in life_cycle_paras_y_copy.items() 
                                               if k in block}

In [95]:
life_cycle_paras_y_copy_ml

{'risk': {'σ_ψ': 0.15, 'σ_θ': 0.1, 'U2U': 0.18, 'E2E': 0.96},
 'initial condition': {'σ_ψ_init': 0.629,
  'init_b': 0.554,
  'bequest_ratio': 0.0},
 'life-cycle': {'T': 40, 'L': 60, '1-D': 0.994},
 'preference': {'ρ': 2.0, 'β': 0.98},
 'policy': {'λ': 0.0, 'λ_SS': 0.0, 'transfer': 0.0, 'μ': 0.15}}

In [99]:
life_cycle_paras_y_copy_df = pd.DataFrame.from_dict({(i,j): life_cycle_paras_y_copy_ml[i][j] 
                           for i in life_cycle_paras_y_copy_ml.keys() 
                           for j in life_cycle_paras_y_copy_ml[i].keys()},
                       orient='index')

In [103]:
## add another column 

life_cycle_paras_y_copy_df.columns =['values']

In [104]:
life_cycle_paras_y_copy_df

,values
"(risk, σ_ψ)",0.150
"(risk, σ_θ)",0.100
"(risk, U2U)",0.180
"(risk, E2E)",0.960
"(initial condition, σ_ψ_init)",0.629
"(initial condition, init_b)",0.554
"(initial condition, bequest_ratio)",0.000
"(life-cycle, T)",40.000
"(life-cycle, L)",60.000
"(life-cycle, 1-D)",0.994


In [105]:
life_cycle_paras_y_copy_df.to_excel('../Tables/calibration.xlsx')